In [100]:

import pandas as pd
import numpy
import sys
sys.maxsize
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

In [124]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
## parameter class fis parameters
class fis_parameters():
    def __init__(self,n_input=3, n_memb=3, batch_size=16, n_epochs=25, memb_func='gaussian',optimizer='sgd', loss='mse'):
        self.n_input = n_input  # no. of Regressors
        self.n_memb = n_memb  # no. of fuzzy memberships
        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.memb_func = memb_func  # 'gaussian' / 'gbellmf'
        self.optimizer = optimizer   # sgd / adam /
        self.loss = loss     ## mse / mae


# Main Class ANFIS
class ANFIS:
    def __init__(self, n_input, n_memb, batch_size=16, memb_func = 'gaussian', name = 'MyAnfis'):
        self.n = n_input
        self.m = n_memb
        self.batch_size = batch_size
        self.memb_func = memb_func
        input_ = keras.layers.Input(shape=(n_input), name='inputLayer', batch_size = batch_size)
        L1 = FuzzyLayer(n_input, n_memb, memb_func, name='fuzzyLayer')(input_)
        L2 = RuleLayer(n_input, n_memb, name='ruleLayer')(L1)
        L3 = NormLayer(name='normLayer')(L2)
        L4 = DefuzzLayer(n_input, n_memb, name='defuzzLayer')(L3, input_)
        L5 = SummationLayer(name='sumLayer')(L4)
        self.model = keras.Model(inputs=[input_], outputs=[L5], name = name)
        self.update_weights()

    def __call__(self, X):
        return self.model.predict(X, batch_size=16)

    def update_weights(self):
        # premise parameters (mu&sigma for gaussian // a/b/c for bell-shaped)
        if self.memb_func == 'gaussian':
            self.mus , self.sigmas = self.model.get_layer('fuzzyLayer').get_weights()
        elif self.memb_func == 'gbellmf':
            self.a , self.b, self.c = self.model.get_layer('fuzzyLayer').get_weights()
        # consequence parameters
        self.bias, self.weights = self.model.get_layer('defuzzLayer').get_weights()

    def plotmfs(self, show_initial_weights=False):
        n_input = self.n
        n_memb = self.m

        if self.memb_func == 'gaussian':
            mus, sigmas = np.around(self.model.get_layer('fuzzyLayer').get_weights(),2)
            mus, sigmas = mus.reshape((n_memb, n_input, 1)), sigmas.reshape(n_memb , n_input, 1)

            xn = np.linspace(np.min(mus)-2*np.max(abs(sigmas)),np.max(mus)+2*np.max(abs(sigmas)), 100).reshape((1,1,-1))
            xn = np.tile(xn, (n_memb, n_input, 1))

            # broadcast all curves in one array
            memb_curves = np.exp(-np.square((xn-mus))/np.square(sigmas))

            if show_initial_weights:
                mus_init, sigmas_init = np.around(self.init_weights,2)
                mus_init, sigmas_init = mus_init.reshape(n_memb, n_input, 1), sigmas_init.reshape(n_memb, n_input, 1)
                init_curves = np.exp(-np.square((xn-mus_init))/np.square(sigmas_init))

        elif self.memb_func == 'gbellmf':
            a, b, c = np.around(self.model.get_layer('fuzzyLayer').get_weights(),2)
            a, b, c = a.reshape((n_memb, n_input, 1)), b.reshape(n_memb, n_input, 1), c.reshape(n_memb, n_input, 1)

            xn = np.linspace(np.min(c)-2*np.max(abs(a)),np.max(c)+2*np.max(abs(a)), 100).reshape((1,1,-1))
            xn = np.tile(xn, (n_memb, n_input, 1))

            # broadcast all curves in one array
            memb_curves= 1/(1+np.square((xn-c)/a)**b)

            if show_initial_weights:
                a_init, b_init, c_init = np.around(self.init_weights,2)
                a_init, b_init, c_init = a_init.reshape((n_memb, n_input, 1)), b_init.reshape(n_memb, n_input, 1), c_init.reshape(n_memb, n_input, 1)
                init_curves = 1/(1+np.square((xn-c_init)/a_init)**b_init)

        elif self.memb_func == 'sigmoid':
            gammas, c = np.around(self.model.get_layer('fuzzyLayer').get_weights(),2)
            gammas, c = gammas.reshape((n_memb, n_input, 1)), c.reshape(n_memb , n_input, 1)

            xn = np.linspace(np.min(c)-2*np.max(abs(c)),np.max(c)+2*np.max(abs(c)), 100).reshape((1,1,-1))  #TODO: change confidence bands
            xn = np.tile(xn, (n_memb, n_input, 1))

            # broadcast all curves in one array
            memb_curves = 1 / (1 + np.exp(-gammas*(xn-c)))

            if show_initial_weights:
                gammas_init, c_init = np.around(self.init_weights,2)
                gammas_init, c_init = gammas_init.reshape(n_memb, n_input, 1), c_init.reshape(n_memb, n_input, 1)
                init_curves = 1 / (1 + np.exp(-gammas_init*(xn-c_init)))

        fig, axs = plt.subplots(nrows=n_input, ncols=1, figsize=(8, self.n*3))
        fig.suptitle('Membership functions', size=16)
        for n in range(self.n):
            axs[n].grid(True)
            axs[n].set_title(f'Input {n+1}')
            for m in range(self.m):
                axs[n].plot(xn[m,n,:], memb_curves[m,n,:])

        if show_initial_weights: # plot initial membership curve
            for n in range(self.n):
                axs[n].set_prop_cycle(None) # reset color cycle
                for m in range(self.m):
                    axs[n].plot(xn[m,n,:], init_curves[m,n,:], '--', alpha=.5)
        plt.show()


    def fit(self, X, y, **kwargs):
        # save initial weights in the anfis class
        self.init_weights = self.model.get_layer('fuzzyLayer').get_weights()

        # fit model & update weights in the anfis class
        history = self.model.fit(X,y, **kwargs)
        self.update_weights()

        # clear the graphs
        tf.keras.backend.clear_session()

        return history

    def get_memberships(self, Xs):
        intermediate_layer_model = keras.Model(inputs = self.model.input,
                                               outputs = self.model.get_layer('normLayer').output)
        intermediate_output = intermediate_layer_model.predict(Xs)

        return intermediate_output



# Custom weight initializer
def equally_spaced_initializer(shape, minval=-1.5, maxval=1.5, dtype=tf.float32):
    """
    Custom weight initializer:
        euqlly spaced weights along an operating range of [minval, maxval].
    """
    linspace = tf.reshape(tf.linspace(minval, maxval, shape[0]),
                          (-1,1))
    return tf.Variable(tf.tile(linspace, (1,shape[1])))



# Layer 1
class FuzzyLayer(keras.layers.Layer):
    def __init__(self, n_input, n_memb, memb_func='gaussian', **kwargs):
        super(FuzzyLayer, self).__init__(**kwargs)
        self.n = n_input
        self.m = n_memb
        self.memb_func = memb_func

    def build(self, batch_input_shape):
        self.batch_size = batch_input_shape[0]

        if self.memb_func == 'gbellmf':
            self.a = self.add_weight(name='a',
                            shape=(self.m, self.n),
                            initializer = keras.initializers.RandomUniform(minval=.7, maxval=1.3, seed=1),
                            #initializer = 'ones',
                            trainable=True)
            self.b = self.add_weight(name='b',
                            shape=(self.m, self.n),
                            initializer = keras.initializers.RandomUniform(minval=.7, maxval=1.3, seed=1),
                            #initializer = 'ones',
                            trainable=True)
            self.c = self.add_weight(name='c',
                            shape=(self.m, self.n),
                            initializer = equally_spaced_initializer,
                            #initializer = keras.initializers.RandomUniform(minval=-1.5, maxval=1.5, seed=1),
                            #initializer = 'zeros',
                            trainable=True)

        elif self.memb_func == 'gaussian':
            self.mu = self.add_weight(name='mu',
                            shape=(self.m, self.n),
                            initializer = equally_spaced_initializer,
                            #initializer = keras.initializers.RandomUniform(minval=-1.5, maxval=1.5, seed=1),
                            #initializer = 'zeros',
                            trainable=True)
            self.sigma = self.add_weight(name='sigma',
                            shape=(self.m, self.n),
                            initializer = keras.initializers.RandomUniform(minval=.7, maxval=1.3, seed=1),
                            #initializer = 'ones',
                            trainable=True)

        elif self.memb_func == 'sigmoid':
            self.gamma = self.add_weight(name = 'gamma',
                            shape=(self.m, self.n),
                            initializer = equally_spaced_initializer, #'ones',
                            trainable = True)

            self.c = self.add_weight(name = 'c',
                            shape=(self.m, self.n),
                            initializer = equally_spaced_initializer, #'ones',
                            trainable = True)

        super(FuzzyLayer, self).build(batch_input_shape)  # Be sure to call this at the end

    def call(self, x_inputs):
        if self.memb_func == 'gbellmf':
            Layer1 = 1/(1+
                   tf.math.pow(
                        tf.square(tf.subtract(
                           tf.reshape(
                               tf.tile(x_inputs, (1, self.m)), (-1, self.m, self.n))
                           ,self.c
                           ) / self.a)
                        , self.b)
                       )
        elif self.memb_func == 'gaussian':
            Layer1 = tf.exp(-1*
                tf.square(tf.subtract(
                    tf.reshape(
                        tf.tile(x_inputs, (1, self.m)), (-1, self.m, self.n))
                    ,self.mu
                    )) / tf.square(self.sigma))

        elif self.memb_func == 'sigmoid':
            Layer1 = tf.math.divide(1,
                        tf.math.exp(-self.gamma*
                            tf.subtract(
                               tf.reshape(
                                   tf.tile(x_inputs, (1, self.m)), (-1, self.m, self.n))
                                            ,self.c)
                                    )
                                    )
        return Layer1  # = fuzzy cluster

    def compute_output_shape(self, batch_input_shape):
        # return ((self.batch_size, self.m, self.n))
        return tf.TensorShape([self.batch_size, self.m, self.n])

# Layer 2
class RuleLayer(keras.layers.Layer):
    def __init__(self, n_input, n_memb, **kwargs):
        super(RuleLayer, self).__init__( **kwargs)
        self.n = n_input
        self.m = n_memb
        self.batch_size = None

    def build(self, batch_input_shape):
        self.batch_size = batch_input_shape[0]
        # self.batch_size = tf.shape(batch_input_shape)[0]
        super(RuleLayer, self).build(batch_input_shape)  # Be sure to call this at the end

    def call(self, input_):
        CP = []
        # a tensor object is not assignable*, so you cannot use it on the left-hand side of an assignment.
        # build a Python list of tensors, and tf.stack() them together at the end of the loop:
        for batch in range(self.batch_size):
            xd_shape = [self.m]
            c_shape = [1]
            cp = input_[batch,:,0]

            for d in range(1,self.n):
                # append shape indizes
                c_shape.insert(0,self.m)
                xd_shape.insert(0,1)
                # get cartesian product for each dimension
                xd = tf.reshape(input_[batch,:,d], (xd_shape))
                c = tf.reshape(cp,(c_shape))
                cp = tf.matmul(c , xd)

            flat_cp = tf.reshape(cp,(1, self.m**self.n))
            CP.append(flat_cp)

        return tf.reshape(tf.stack(CP), (self.batch_size, self.m**self.n))

    def compute_output_shape(self, batch_input_shape):
        if self.n == 1:
            return tf.TensorShape([self.batch_size, self.m])
        else:
            return tf.TensorShape([self.batch_size, self.m** self.n])

# Layer 3
class NormLayer(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__( **kwargs)

    def call(self, fire):
        w_sum = tf.reshape(tf.reduce_sum(fire, axis=1), (-1,1))
        w_norm = fire / w_sum
        return w_norm

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

# Layer 4
class DefuzzLayer(keras.layers.Layer):
    def __init__(self, n_input, n_memb, **kwargs):
        super().__init__( **kwargs)
        self.n = n_input
        self.m = n_memb

    def build(self, batch_input_shape):
        self.CP_bias = self.add_weight(name='Consequence_bias',
                                                 shape=(1, self.m **self.n),
                                                 initializer = keras.initializers.RandomUniform(minval=-2, maxval=2),
                                                 # initializer = 'ones',
                                                 trainable=True)
        self.CP_weight = self.add_weight(name='Consequence_weight',
                                            shape=(self.n, self.m ** self.n),
                                            initializer = keras.initializers.RandomUniform(minval=-2, maxval=2),
                                            # initializer = 'ones',
                                            trainable=True)

    def call(self, w_norm, Xs):

        Layer4=tf.multiply(w_norm,
                           tf.matmul(Xs, self.CP_weight) + self.CP_bias)
        return Layer4  # Defuzzyfied Layer

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

# Layer 5
class SummationLayer(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__( **kwargs)

    def build(self, batch_input_shape):
        self.batch_size = batch_input_shape[0]
        #self.batch_size = tf.shape(batch_input_shape)[0]
        super(SummationLayer, self).build(batch_input_shape)  # Be sure to call this at the end

    def call(self, Layer4):
        output = tf.reduce_sum(Layer4, axis=1)
        output = tf.reshape(output, (self.batch_size, 1))
        return output # final output

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape([self.batch_size,1])

In [134]:
import numpy as np
%matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy.special

def Func(x, y):
    z = (np.sin(x) * np.exp((1 - np.cos(y)) ** 2) + np.cos(y) * np.exp((1 - np.sin(x)) ** 2) + (x - y) ** 2)
    return z

Using matplotlib backend: Qt5Agg


In [155]:
# set parameters
param = fis_parameters(
        n_input = 2,                # no. of Regressors
        n_memb = 2,                 # no. of fuzzy memberships
        batch_size = 16,            # 16 / 32 / 64 / ...
        memb_func = 'gaussian',      # 'gaussian' / 'gbellmf' / 'sigmoid'
        optimizer = 'adam',          # sgd / adam / ...
        loss = 'mse',               # mse / mae / huber_loss / mean_absolute_percentage_error / ...
        n_epochs = 1000              # 10 / 25 / 50 / 100 / ...
        )
    # create random data
#X_train = np.random.rand(param.batch_size*5, param.n_input),
#X_test = np.random.rand(param.batch_size*2, param.n_input)
#y_train = np.random.rand(param.batch_size*5,1),
#y_test = np.random.rand(param.batch_size*2, 1)

X_train = []
y_train = []
X_test = []
y_test = []

d = 10

for x in range(-5*d, 5*d):
    for y in range(-5*d, 5*d):
        X_train += [[x/d, y/d]]
        y_train += [Func(x/d, y/d)]
        
for x in range(-5*d, 5*d):
    for y in range(-5*d, 5*d):
        X_test += [[x/d, y/d]]
        y_test += [Func(x/d, y/d)]
        
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)


fis = ANFIS(n_input = param.n_input,
                    n_memb = param.n_memb,
                    batch_size = param.batch_size,
                    memb_func = param.memb_func,
                    name = 'myanfis'
                    )

    # compile model
fis.model.compile(optimizer=param.optimizer,
                loss=param.loss
                #,metrics=['mse']  # ['mae', 'mse']
                )

    # fit model
history = fis.fit(X_train, y_train,
                epochs=param.n_epochs,
                batch_size=param.batch_size,
                validation_data = (X_test, y_test),
                # callbacks = [tensorboard_callback]  # for tensorboard
                )

    # eval model
import pandas as pd
fis.plotmfs(show_initial_weights=True)

loss_curves = pd.DataFrame(history.history)
loss_curves.plot(figsize=(8, 5))

fis.model.summary()

# get premise parameters
premise_parameters = fis.model.get_layer('fuzzyLayer').get_weights()       # alternative

    # get consequence paramters
bias = fis.bias
weights = fis.weights
    # conseq_parameters = fis.model.get_layer('defuzzLayer').get_weights()       # alternative

Epoch 1/1000
625/625 [==============================] - 2s 2ms/step - loss: 1258.2657 - val_loss: 1211.4744
Epoch 2/1000
625/625 [==============================] - 1s 2ms/step - loss: 1173.0649 - val_loss: 1136.3475
Epoch 3/1000
625/625 [==============================] - 1s 2ms/step - loss: 1103.4919 - val_loss: 1071.9127
Epoch 4/1000
625/625 [==============================] - 1s 2ms/step - loss: 1043.6302 - val_loss: 1016.4278
Epoch 5/1000
625/625 [==============================] - 1s 2ms/step - loss: 992.4922 - val_loss: 969.5280
Epoch 6/1000
625/625 [==============================] - 1s 2ms/step - loss: 948.8751 - val_loss: 929.2832
Epoch 7/1000
625/625 [==============================] - 1s 2ms/step - loss: 912.2632 - val_loss: 895.9133
Epoch 8/1000
625/625 [==============================] - 1s 2ms/step - loss: 881.5729 - val_loss: 867.8094
Epoch 9/1000
625/625 [==============================] - 1s 2ms/step - loss: 855.8427 - val_loss: 844.4135
Epoch 10/1000
625/625 [===============

625/625 [==============================] - 1s 2ms/step - loss: 585.0372 - val_loss: 584.3453
Epoch 78/1000
625/625 [==============================] - 1s 2ms/step - loss: 583.7117 - val_loss: 582.8706
Epoch 79/1000
625/625 [==============================] - 1s 2ms/step - loss: 582.2875 - val_loss: 581.6581
Epoch 80/1000
625/625 [==============================] - 1s 2ms/step - loss: 581.0604 - val_loss: 580.2413
Epoch 81/1000
625/625 [==============================] - 1s 2ms/step - loss: 579.7731 - val_loss: 578.9883
Epoch 82/1000
625/625 [==============================] - 1s 2ms/step - loss: 578.5133 - val_loss: 577.7099
Epoch 83/1000
625/625 [==============================] - 1s 2ms/step - loss: 577.2676 - val_loss: 576.4636
Epoch 84/1000
625/625 [==============================] - 1s 2ms/step - loss: 576.0007 - val_loss: 575.2540
Epoch 85/1000
625/625 [==============================] - 1s 2ms/step - loss: 574.8062 - val_loss: 574.0360
Epoch 86/1000
625/625 [============================

625/625 [==============================] - 1s 2ms/step - loss: 523.1757 - val_loss: 522.8636
Epoch 154/1000
625/625 [==============================] - 1s 2ms/step - loss: 522.7446 - val_loss: 522.4133
Epoch 155/1000
625/625 [==============================] - 1s 2ms/step - loss: 522.2875 - val_loss: 521.8921
Epoch 156/1000
625/625 [==============================] - 1s 2ms/step - loss: 521.8629 - val_loss: 521.4583
Epoch 157/1000
625/625 [==============================] - 1s 2ms/step - loss: 521.4291 - val_loss: 521.0397
Epoch 158/1000
625/625 [==============================] - 1s 2ms/step - loss: 520.9592 - val_loss: 520.5776
Epoch 159/1000
625/625 [==============================] - 1s 2ms/step - loss: 520.5553 - val_loss: 520.1411
Epoch 160/1000
625/625 [==============================] - 1s 2ms/step - loss: 520.1006 - val_loss: 519.7669
Epoch 161/1000
625/625 [==============================] - 1s 2ms/step - loss: 519.7368 - val_loss: 519.3039
Epoch 162/1000
625/625 [===================

Epoch 229/1000
625/625 [==============================] - 1s 2ms/step - loss: 492.2621 - val_loss: 493.5402
Epoch 230/1000
625/625 [==============================] - 1s 2ms/step - loss: 492.1229 - val_loss: 493.4106
Epoch 231/1000
625/625 [==============================] - 1s 2ms/step - loss: 491.2975 - val_loss: 500.6134
Epoch 232/1000
625/625 [==============================] - 1s 2ms/step - loss: 492.0393 - val_loss: 491.1882
Epoch 233/1000
625/625 [==============================] - 1s 2ms/step - loss: 491.6190 - val_loss: 491.1480
Epoch 234/1000
625/625 [==============================] - 1s 2ms/step - loss: 491.6003 - val_loss: 490.1815
Epoch 235/1000
625/625 [==============================] - 1s 2ms/step - loss: 491.0247 - val_loss: 490.1818
Epoch 236/1000
625/625 [==============================] - 1s 2ms/step - loss: 490.7576 - val_loss: 489.8186
Epoch 237/1000
625/625 [==============================] - 1s 2ms/step - loss: 490.2968 - val_loss: 490.7993
Epoch 238/1000
625/625 [====

Epoch 305/1000
625/625 [==============================] - 1s 2ms/step - loss: 479.3123 - val_loss: 478.7980
Epoch 306/1000
625/625 [==============================] - 1s 2ms/step - loss: 478.6333 - val_loss: 479.0485
Epoch 307/1000
625/625 [==============================] - 1s 2ms/step - loss: 479.1873 - val_loss: 478.7310
Epoch 308/1000
625/625 [==============================] - 1s 2ms/step - loss: 479.3586 - val_loss: 478.1764
Epoch 309/1000
625/625 [==============================] - 1s 2ms/step - loss: 479.5383 - val_loss: 478.4856
Epoch 310/1000
625/625 [==============================] - 1s 2ms/step - loss: 479.2465 - val_loss: 478.5356
Epoch 311/1000
625/625 [==============================] - 1s 2ms/step - loss: 478.9214 - val_loss: 477.8669
Epoch 312/1000
625/625 [==============================] - 1s 2ms/step - loss: 478.9430 - val_loss: 477.9029
Epoch 313/1000
625/625 [==============================] - 1s 2ms/step - loss: 479.1751 - val_loss: 477.9055
Epoch 314/1000
625/625 [====

...

Epoch 685/1000
625/625 [==============================] - 1s 2ms/step - loss: 440.5796 - val_loss: 439.6428
Epoch 686/1000
625/625 [==============================] - 1s 2ms/step - loss: 440.4877 - val_loss: 440.5129
Epoch 687/1000
625/625 [==============================] - 1s 2ms/step - loss: 440.2250 - val_loss: 440.0110
Epoch 688/1000
625/625 [==============================] - 1s 2ms/step - loss: 440.4684 - val_loss: 439.8360
Epoch 689/1000
625/625 [==============================] - 2s 2ms/step - loss: 439.9080 - val_loss: 439.2960
Epoch 690/1000
625/625 [==============================] - 1s 2ms/step - loss: 439.9869 - val_loss: 439.3163
Epoch 691/1000
625/625 [==============================] - 1s 2ms/step - loss: 439.7991 - val_loss: 439.5870
Epoch 692/1000
625/625 [==============================] - 1s 2ms/step - loss: 439.8218 - val_loss: 439.1187
Epoch 693/1000
625/625 [==============================] - 1s 2ms/step - loss: 439.4293 - val_loss: 439.2230
Epoch 694/1000
625/625 [====

Epoch 761/1000
625/625 [==============================] - 1s 2ms/step - loss: 434.7596 - val_loss: 435.6054
Epoch 762/1000
625/625 [==============================] - 1s 2ms/step - loss: 435.0154 - val_loss: 434.4456
Epoch 763/1000
625/625 [==============================] - 1s 2ms/step - loss: 434.9980 - val_loss: 434.0967
Epoch 764/1000
625/625 [==============================] - 1s 2ms/step - loss: 434.7640 - val_loss: 434.4673
Epoch 765/1000
625/625 [==============================] - 1s 2ms/step - loss: 434.8476 - val_loss: 434.5860
Epoch 766/1000
625/625 [==============================] - 1s 2ms/step - loss: 434.6589 - val_loss: 434.4048
Epoch 767/1000
625/625 [==============================] - 1s 2ms/step - loss: 434.6183 - val_loss: 433.8635
Epoch 768/1000
625/625 [==============================] - 1s 2ms/step - loss: 434.7202 - val_loss: 435.0379
Epoch 769/1000
625/625 [==============================] - 1s 2ms/step - loss: 434.8665 - val_loss: 433.6125
Epoch 770/1000
625/625 [====

Epoch 837/1000
625/625 [==============================] - 1s 2ms/step - loss: 430.6559 - val_loss: 430.0206
Epoch 838/1000
625/625 [==============================] - 1s 2ms/step - loss: 430.3552 - val_loss: 429.4695
Epoch 839/1000
625/625 [==============================] - 1s 2ms/step - loss: 430.5240 - val_loss: 429.8809
Epoch 840/1000
625/625 [==============================] - 1s 2ms/step - loss: 430.3467 - val_loss: 429.2860
Epoch 841/1000
625/625 [==============================] - 1s 2ms/step - loss: 429.8545 - val_loss: 429.1744
Epoch 842/1000
625/625 [==============================] - 1s 2ms/step - loss: 429.5105 - val_loss: 429.2553
Epoch 843/1000
625/625 [==============================] - 1s 2ms/step - loss: 430.1172 - val_loss: 429.1915
Epoch 844/1000
625/625 [==============================] - 1s 2ms/step - loss: 429.7013 - val_loss: 429.4795
Epoch 845/1000
625/625 [==============================] - 1s 2ms/step - loss: 429.9301 - val_loss: 428.8975
Epoch 846/1000
625/625 [====

Epoch 913/1000
625/625 [==============================] - 1s 2ms/step - loss: 425.8354 - val_loss: 425.2054
Epoch 914/1000
625/625 [==============================] - 1s 2ms/step - loss: 426.0456 - val_loss: 425.1134
Epoch 915/1000
625/625 [==============================] - 1s 2ms/step - loss: 426.2199 - val_loss: 425.4719
Epoch 916/1000
625/625 [==============================] - 1s 2ms/step - loss: 425.5222 - val_loss: 425.0550
Epoch 917/1000
625/625 [==============================] - 1s 2ms/step - loss: 425.8841 - val_loss: 425.1843
Epoch 918/1000
625/625 [==============================] - 1s 2ms/step - loss: 425.5645 - val_loss: 425.8625
Epoch 919/1000
625/625 [==============================] - 1s 2ms/step - loss: 426.1813 - val_loss: 424.8706
Epoch 920/1000
625/625 [==============================] - 1s 2ms/step - loss: 425.5682 - val_loss: 425.6050
Epoch 921/1000
625/625 [==============================] - 1s 2ms/step - loss: 425.6942 - val_loss: 424.7816
Epoch 922/1000
625/625 [====

Epoch 989/1000
625/625 [==============================] - 1s 2ms/step - loss: 422.1420 - val_loss: 421.7935
Epoch 990/1000
625/625 [==============================] - 1s 2ms/step - loss: 422.3930 - val_loss: 421.5430
Epoch 991/1000
625/625 [==============================] - 1s 2ms/step - loss: 422.4421 - val_loss: 421.4875
Epoch 992/1000
625/625 [==============================] - 1s 2ms/step - loss: 422.2074 - val_loss: 421.7827
Epoch 993/1000
625/625 [==============================] - 1s 2ms/step - loss: 422.0463 - val_loss: 421.3877
Epoch 994/1000
625/625 [==============================] - 1s 2ms/step - loss: 422.5646 - val_loss: 421.3799
Epoch 995/1000
625/625 [==============================] - 1s 2ms/step - loss: 422.2714 - val_loss: 421.4334
Epoch 996/1000
625/625 [==============================] - 1s 2ms/step - loss: 422.0880 - val_loss: 422.6993
Epoch 997/1000
625/625 [==============================] - 1s 2ms/step - loss: 422.3510 - val_loss: 421.3490
Epoch 998/1000
625/625 [====

In [152]:
fis(X_train)

array([[ -5.5429006],
       [ -4.987119 ],
       [ -4.4311514],
       ...,
       [-23.409588 ],
       [-20.772228 ],
       [-18.476166 ]], dtype=float32)

In [156]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter(X_test[:, 0], X_test[:, 1], y_test, c='blue', marker='o')
result_set = fis(X_train)
ax.scatter(X_test[:, 0], X_test[:, 1], result_set, c='red', marker='o')